In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from code import *
from useful_functions import *

$x^{(k+1)}=Ax^{(k)}-\alpha y^{(k)} $

$V_{k+1}= AV_{k}$

$y^{(k+1)}=Ay^{(k)}+D_{k+1}^{-1}(g^{(k+1)}-g^{(k)} ) $

其中, $D_{k}=\text{Diag}(V_k)^{-1} $

计算最后的平均梯度用的是$\mathbf{1}_n\pi_A^T\mathbf{x}^{(k)}$，不过和使用$\frac{1}{n}\mathbf{1}_n\mathbf{1}_n^T\mathbf{x}^{(k)}$“应该”区别不大

In [ ]:
def pull_sum_original(A, init_x, h_data, y_data, grad_func, grad_f_bar_func, d=784, rho=0.1, lr=0.1,sigma_n=0.1, max_it=200, mg=1):
    n = A.shape[0]
    h, y = h_data, y_data
    x = init_x
    g = grad_func(x, y, h, rho=rho).reshape(x.shape)+sigma_n/mg*np.random.normal(size=(n,d))
    v = g 
    pi=get_left_perron(A).reshape((1,n))
    A0=np.ones((n, 1))@pi@A
    B=get_B(A,9,n)
    correction_vec=np.ones(n)
    gradient_f_bar_x_val = grad_f_bar_func((A0@x).reshape(x.shape), y, h, rho=rho)
    gradient_history=[np.linalg.norm(gradient_f_bar_x_val)]
    v_history=[np.linalg.norm(v)]
    for i in range(max_it):
        
        correction_vec=A.T@correction_vec 
        x = A @ x - lr * np.diag(1/correction_vec) @ v 
        pre_g = g
        gradient=grad_func(x, y, h, rho=rho)
        g = gradient.reshape(x.shape)+sigma_n/mg*np.random.normal(size=(n,d))
        v = B @ v + g - pre_g
        g1=grad_f_bar_func((A0@x).reshape(x.shape), y, h, rho=rho)
        gradient_history.append(np.linalg.norm(g1))
        v_history.append(np.linalg.norm(np.diag(1/correction_vec) @v))

    return gradient_history ,v_history

In [ ]:
def pull_sum(
    A,
    init_x,
    h_data,
    y_data,
    grad_func,
    grad_f_bar_func,
    d=784,
    rho=0.1,
    lr=0.1,
    sigma_n=0.1,
    max_it=200,
    mg=1,
    marm_up=10,
):
    """新的 pull sum 算法"""
    n = A.shape[0]
    h, y = h_data, y_data
    x = init_x
    g = grad_func(x, y, h, rho=rho).reshape(x.shape) + sigma_n / mg * np.random.normal(
        size=(n, d)
    )
    V = np.eye(n)
    v = g

    pi = get_left_perron(A).reshape((1, n))
    A0 = np.ones((n, 1)) @ pi @ A
    gradient_f_bar_x_val = grad_f_bar_func((A0 @ x).reshape(x.shape), y, h, rho=rho)
    gradient_history = [np.linalg.norm(gradient_f_bar_x_val)]
    v_history = [np.linalg.norm(v)]

    for i in range(max_it):
        if i < marm_up:
            x = A @ x - lr / 100 * v
        else:
            x = A @ x - lr * v
        pre_g = g
        g = grad_func(x, y, h, rho=rho).reshape(
            x.shape
        ) + sigma_n / mg * np.random.normal(size=(n, d))
        V = A @ V
        D = np.diag(1 / np.diag(V))
        v = A @ v + D @ (g - pre_g)

        gradient_history.append(np.linalg.norm(g))
        v_history.append(np.linalg.norm(v))
    return gradient_history, v_history

$W=I$

Ls 跟踪矩阵$A$的对角线元素的最小值的“倒数”，已知矩阵$A$的对角元会收敛到$\pi_A$

$x^{(k+1)}=Ax^{(k)}-lr*v^{(k)}$

$v^{(k+1)}=Av^{(k)}+\text{Diag}(A^{k+1})^{-1}g^{(k+1)}-\text{Diag}(A^{k})^{-1}g^{(k)}$

In [ ]:
def pull_diag(
    A,
    init_x,
    h_data,
    y_data,
    grad_func,
    grad_f_bar_func,
    d=784,
    rho=0.1,
    lr=0.1,
    sigma_n=0.1,
    max_it=200,
    mg=1,
):
    """实际上是pull diag算法"""
    n = A.shape[0]
    h, y = h_data, y_data
    x = init_x
    W = np.eye(n)
    g = grad_func(x, y, h, rho=rho).reshape(x.shape) + sigma_n / mg * np.random.normal(
        size=(n, d)
    )
    w = np.linalg.inv(np.diag(np.diag(W))) @ g
    v = g
    pi = get_left_perron(A).reshape((1, n))
    A0 = np.ones((n, 1)) @ pi @ A
    gradient_f_bar_x_val = grad_f_bar_func((A0 @ x).reshape(x.shape), y, h, rho=rho)
    gradient_history = [np.linalg.norm(gradient_f_bar_x_val)]
    Ls = [1 / min(np.diag(A))]
    v_history = [np.linalg.norm(v)]
    for i in range(max_it):
        W = A @ W
        x = A @ x - lr * v
        gradient = grad_func(x, y, h, rho=rho)
        g = gradient.reshape(x.shape) + sigma_n / mg * np.random.normal(size=(n, d))
        v = A @ v + np.linalg.inv(np.diag(np.diag(W))) @ g - w
        w = (
            np.linalg.inv(np.diag(np.diag(W))) @ g
        )  # 这一步计算的w是下一步用到的w，因此程序没有问题
        g1 = grad_f_bar_func((A0 @ x).reshape(x.shape), y, h, rho=rho)
        gradient_history.append(np.linalg.norm(g1))
        Ls.append(1 / min(np.diag(W)))
        v_history.append(np.linalg.norm(v))

    return gradient_history, v_history, max(Ls)

correction vector: $c^{(k)}=(A^T)^k\mathbf{1}_n\to n\pi_A$

$x^{(k+1)}= Ax^{(k)}-lr*\text{Diag}(c^{(k)})^{-1}v^{(k)}$

$v^{(k+1)}=\textcolor{red}{A}v^{(k)}+g^{(k+1)}-g^{(k)}$

In [7]:
def pull_sum_AGT(A, init_x, h_data, y_data, grad_func, loss_func, grad_f_bar_func, d=784, L=1, rho=0.1, lr=0.1,sigma_n=0.1, max_it=200, mg=1, decay=1):
    n = A.shape[0]
    h, y = h_data, y_data
    x = init_x
    g = grad_func(x, y, h, rho=rho).reshape(x.shape)+sigma_n/mg*np.random.normal(size=(n,d))
    v = g 
    pi=get_left_perron(A).reshape((1,n))
    A0=np.ones((n, 1))@pi@A
    
    correction_vec=np.ones(n)
    gradient_f_bar_x_val = grad_f_bar_func((A0@x).reshape(x.shape), y, h, rho=rho)
    gradient_history=[np.linalg.norm(gradient_f_bar_x_val)]
    v_history=[np.linalg.norm(v)]
    for i in range(max_it):
        
        correction_vec=A.T@correction_vec 
        x = A @ x - lr * np.diag(1/correction_vec) @ v 
        pre_g = g
        gradient=grad_func(x, y, h, rho=rho)
        g = gradient.reshape(x.shape)+sigma_n/mg*np.random.normal(size=(n,d))
        v = A @ v + g - pre_g
        g1=grad_f_bar_func((A0@x).reshape(x.shape), y, h, rho=rho)
        gradient_history.append(np.linalg.norm(g1))
        v_history.append(np.linalg.norm(np.diag(1/correction_vec) @v))

    return gradient_history ,v_history

correction vector: $c^{(k)}=(A^T)^k\mathbf{1}_n\to n\pi_A$

$\textcolor{blue}{B={get_B(A,9,n)}}$

$x^{(k+1)}= Ax^{(k)}-lr*\text{Diag}(c^{(k)})^{-1}v^{(k)}$

$v^{(k+1)}=\textcolor{blue}{B}v^{(k)}+g^{(k+1)}-g^{(k)}$

In [8]:
def pull_diag_BGT(A, init_x, h_data, y_data, grad_func, loss_func, grad_f_bar_func, d=784, L=1, rho=0.1, lr=0.1,sigma_n=0.1, max_it=200, mg=1, decay=1):
    n = A.shape[0]
    h, y = h_data, y_data
    x = init_x
    W = np.eye(n)
    g = grad_func(x, y, h, rho=rho).reshape(x.shape)+sigma_n/mg*np.random.normal(size=(n,d))
    w = np.linalg.inv(np.diag(np.diag(W)))@g
    v= g
    pi=get_left_perron(A).reshape((1,n))
    A0=np.ones((n, 1))@pi@A
    gradient_f_bar_x_val = grad_f_bar_func((A0@x).reshape(x.shape), y, h, rho=rho)
    gradient_history=[np.linalg.norm(gradient_f_bar_x_val)]
    Ls=[1/min(np.diag(A))]
    v_history=[np.linalg.norm(v)]
    B=get_B(A,9,n)
    for i in range(max_it):
        W=A@W
        x = A @ x - lr *  v
        gradient=grad_func(x, y, h, rho=rho)
        g = gradient.reshape(x.shape)+sigma_n/mg*np.random.normal(size=(n,d))
        v = B @ v + np.linalg.inv(np.diag(np.diag(W)))@g - w
        w = np.linalg.inv(np.diag(np.diag(W)))@g
        g1=grad_f_bar_func((A0@x).reshape(x.shape), y, h, rho=rho)
        gradient_history.append(np.linalg.norm(g1))
        Ls.append(1/min(np.diag(W)))
        v_history.append(np.linalg.norm(v))

    return gradient_history, v_history

In [9]:
def compute_kappa_row(A):
    pi=get_left_perron(A)
    return np.max(pi)/np.min(pi)

def compute_kappa_col(B):
    pi=get_right_perron(B)
    return np.max(pi)/np.min(pi)

#计算第二大特征值的模长
def compute_2st_eig_value(A):
    return abs(np.linalg.eigvals(A)[1])

#计算beta
import numpy as np
import networkx as nx
from mpmath import mp

def compute_beta_row(A, precision=64):
    mp.dps = precision  # 设置计算精度
    n = A.shape[0]
    pi = get_left_perron(A)
    one = np.ones(n)
    if not nx.is_strongly_connected(nx.DiGraph(A)):
        print("不是强联通")
    matrix = A - np.outer(one, pi)
    diag1 = np.diag(np.sqrt(pi))
    diag1_inverse = np.diag(1 / np.sqrt(pi))
    result = np.linalg.norm(diag1 @ matrix @ diag1_inverse, 2)
    return min(result, 1)  # 裁剪结果不超过1

def compute_beta_col(B, precision=64):
    mp.dps = precision  # 设置计算精度
    n = B.shape[0]
    pi = get_right_perron(B)
    one = np.ones(n)
    if not nx.is_strongly_connected(nx.DiGraph(B)):
        print("不是强联通")
    matrix = B - np.outer(pi, one)
    diag1 = np.diag(np.sqrt(pi))
    diag1_inverse = np.diag(1 / np.sqrt(pi))
    result = np.linalg.norm(diag1_inverse @ matrix @ diag1, 2)
    return min(result, 1)  # 裁剪结果不超过1


def compute_S_A_row(A):
    kappa=compute_kappa_row(A)
    beta=compute_beta_row(A)
    n=A.shape[0]
    output=2*np.sqrt(n)*(1+np.log(kappa))/(1-beta)
    return output

def compute_S_B_col(B):
    kappa=compute_kappa_col(B)
    beta=compute_beta_col(B)
    n=B.shape[0]
    output=2*np.sqrt(n)*(1+np.log(kappa))/(1-beta)
    return output

def show_row(A):
    print("A的第二大特征值:",compute_2st_eig_value(A))
    print("A的beta:",compute_beta_row(A))
    print("A的spectral gap:",1-compute_beta_row(A))
    print("A的kappa:",compute_kappa_row(A))
    print("S_A是:",compute_S_A_row(A),"\n")